In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Carregar dataset CIFAR-10
(train_ds, val_ds, test_ds), ds_info = tfds.load(
    "cifar10",
    split=["train[:80%]", "train[80%:]", "test"],
    as_supervised=True,
    with_info=True
)

# Definir parâmetros
IMG_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.0AIZV5_3.0.2/cifar10-train.tfrecord*...:   0%|         …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.0AIZV5_3.0.2/cifar10-test.tfrecord*...:   0%|          …

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [2]:
# Funções de preprocessamento e augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) # resize da imagem para o MobileNetV2
    image = preprocess_input(image)  # normaliza com função própria do MobileNetV2
    return image, label

In [3]:
# Preparar datasets
def prepare_dataset(ds, training=False):
    ds = ds.map(preprocess, num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=AUTOTUNE)
        ds = ds.shuffle(1000)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

train_ds = prepare_dataset(train_ds, training=True)
val_ds = prepare_dataset(val_ds)
test_ds = prepare_dataset(test_ds)

In [4]:
# Carregar modelo pré-treinado
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                         include_top=False,
                         weights="imagenet")
base_model.trainable = False  # congelado

# Construir modelo final
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = models.Model(inputs, outputs)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
# Compilar
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Treinar (feature extraction)
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10)

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 448s 336ms/step - accuracy: 0.5758 - loss: 1.2327 - val_accuracy: 0.7874 - val_loss: 0.6244
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 400s 313ms/step - accuracy: 0.7182 - loss: 0.8193 - val_accuracy: 0.8199 - val_loss: 0.5227
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 401s 314ms/step - accuracy: 0.7283 - loss: 0.8016 - val_accuracy: 0.8259 - val_loss: 0.5050
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 442s 313ms/step - accuracy: 0.7335 - loss: 0.7862 - val_accuracy: 0.8245 - val_loss: 0.5118
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 399s 312ms/step - accuracy: 0.7331 - loss: 0.7862 - val_accuracy: 0.8208 - val_loss: 0.5205
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 400s 312ms/step - accuracy: 0.7347 - loss: 0.7782 - val_accuracy: 0.8168 - val_loss: 0.5431
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 442s 313ms/step - accuracy: 0.7342 - loss: 0.7915 - val_accuracy: 0.8233 - val_loss: 0.5241
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 403s 314ms/step - ac

In [6]:
# Fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-50]:  # congela tudo exceto as últimas 50 camadas
    layer.trainable = False

# Compilar denovo
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # LR bem menor
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Continuar treino (fine-tuning)
history_fine = model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=10)

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 454s 338ms/step - accuracy: 0.6639 - loss: 1.0853 - val_accuracy: 0.8358 - val_loss: 0.5024
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 423s 331ms/step - accuracy: 0.7770 - loss: 0.6558 - val_accuracy: 0.8567 - val_loss: 0.4207
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 425s 332ms/step - accuracy: 0.8005 - loss: 0.5703 - val_accuracy: 0.8695 - val_loss: 0.3717
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 422s 330ms/step - accuracy: 0.8305 - loss: 0.5002 - val_accuracy: 0.8792 - val_loss: 0.3453
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 424s 332ms/step - accuracy: 0.8408 - loss: 0.4581 - val_accuracy: 0.8912 - val_loss: 0.3120
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 423s 331ms/step - accuracy: 0.8566 - loss: 0.4148 - val_accuracy: 0.8992 - val_loss: 0.2912
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 424s 332ms/step - accuracy: 0.8657 - loss: 0.3914 - val_accuracy: 0.9017 - val_loss: 0.2800
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 425s 333ms/step - ac

In [7]:
# Avaliar no conjunto de teste
test_loss, test_acc = model.evaluate(test_ds)
print(f"\nAcurácia no teste: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.9089 - loss: 0.2581

Acurácia no teste: 0.9078
